# Out-of_site Mapping Model
Creates the out-of-site model used for mapping. It creates the ensemble of 20 models and saves the Keras models as HDF5 files. These files can then be loaded and used to generate the LFMC estimates needed to create LFMC maps.

In [ ]:
import os

import initialise
import common
from modelling_functions import create_models
from architecture_out_of_site import model_params
from model_parameters import ModelParams, ExperimentParams
import scenarios

## Input files
Change these settings as required
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted DEM and other auxiliary data for each sample, created by `Extract DEM Data.ipynb`. This includes the labels and pre-processed site location and sampling date data from Globe-LFMC

In [ ]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_365days.csv')
prism_csv = os.path.join(common.DATASETS_DIR, 'prism_365days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_365days.csv')

## Set up model parameters

### Model parameters settings
To find out more about any parameter, run `model_params.help('<parameter>')`. 

In [ ]:
# Customize model parameters
scenarios.common_params(model_params)
model_params['modelName'] = 'out-of-site_map2017'
model_params['description'] = 'Create an ensemble of out-of-site models for 2017 LFMC maps'
model_params['samplesFile'] = aux_csv
model_params['saveModels'] = True       # Save the models to use for map-wide estimates
model_params['splitMethod'] = None
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])

model_params.add_input('optical', {'filename': modis_csv, 'channels': 7})
model_params.add_input('weather', {'filename': prism_csv, 'channels': 7})

model_params

## Build and run the model
Builds and trains the LFMC model. After training the model, several derived models are created and evaluated. The full list of derived models is:
- `base` - The fully trained model
- `merge10` - A model created by merging the last 10 checkpoints. The checkpoints are merged by averaging the corresponding weights from each model.

The model and training statistics are saved to `model_dir`.

In [ ]:
model = create_models(model_params)